In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

print(os.getcwd())

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.
/rds/homes/g/gaddcz/Projects/CPRD/examples/modelling/SurvivEHR/notebooks/CompetingRisk/1_Study3_MultiMorbidity


In [2]:
# from FastEHR.dataloader import *
import FastEHR
from FastEHR.dataloader import FoundationalDataModule

In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import logging
import wandb
import pickle
from hydra import compose, initialize
from omegaconf import OmegaConf

from FastEHR.dataloader import FoundationalDataModule

from CPRD.examples.modelling.SurvivEHR.run_experiment import run
from CPRD.src.models.survival.task_heads.causal import SurvStreamGPTForCausalModelling

import time
import polars as pl
pl.Config.set_tbl_rows(10000)
import pandas as pd
pd.options.display.max_rows = 10000


# TODO: define an env variable to fix for a local hpc environment issue, this shouldn't be needed
%env SLURM_NTASKS_PER_NODE=28   

env: SLURM_NTASKS_PER_NODE=28


# ctd = 0.7236599922180176
# ibs = 0.1104101305535031
# inbll = 0.34205402945594054
plt.close()

current_diagnoses = [0,1,2,3,4,5,6,7,8,9,10]
RMST = [0.7974921060097665,
        0.7807423914145325 , 
        0.762604199063575,
        0.7455309041315031,
        0.7312270840661624,
        0.7175176664645307,
        0.7032747673275282,
        0.6926341798480199,
        0.6706015073433462,
        0.6625103828978652,
        0.6440878911760254]



│     Test:RMST_0 current diagnoses      │           0.7974921060097665           │
│     Test:RMST_1 current diagnosis      │           0.7807423914145325           │
│     Test:RMST_2 current diagnoses      │           0.762604199063575            │
│     Test:RMST_3 current diagnoses      │           0.7455309041315031           │
│     Test:RMST_4 current diagnoses      │           0.7312270840661624           │
│     Test:RMST_5 current diagnoses      │           0.7175176664645307           │
│     Test:RMST_6 current diagnoses      │           0.7032747673275282           │
│     Test:RMST_7 current diagnoses      │           0.6926341798480199           │
│     Test:RMST_8 current diagnoses      │           0.6706015073433462           │
│     Test:RMST_9 current diagnoses      │           0.6625103828978652           │
│     Test:RMST_10 current diagnoses     │           0.6440878911760254           │

obs_MST = [0.8193886293519516,
           0.8064669470307623,
           0.7976435723488524,
           0.7816793664656079,
           0.7664040945019838,
           0.7629594680202728,
           0.7317382655329833,
           0.726904411096962,
           0.7239051935624105,
           0.632658596392031,
           0.7027527945382255]


│ observedTest:RMST_0 current diagnoses  │           0.8193886293519516           │
│ observedTest:RMST_1 current diagnosis  │           0.8064669470307623           │
│ observedTest:RMST_2 current diagnoses  │           0.7976435723488524           │
│ observedTest:RMST_3 current diagnoses  │           0.7816793664656079           │
│ observedTest:RMST_4 current diagnoses  │           0.7664040945019838           │
│ observedTest:RMST_5 current diagnoses  │           0.7629594680202728           │
│ observedTest:RMST_6 current diagnoses  │           0.7317382655329833           │
│ observedTest:RMST_7 current diagnoses  │           0.726904411096962            │
│ observedTest:RMST_8 current diagnoses  │           0.7239051935624105           │
│ observedTest:RMST_9 current diagnoses  │           0.632658596392031            │
│ observedTest:RMST_10 current diagnoses │           0.7027527945382255           │

│               test_loss                │           0.7400515079498291           │

plt.plot(current_diagnoses[:8], RMST[:8], label="SurvivEHR")
plt.plot(current_diagnoses[:8], obs_MST[:8], label="Observed", color='k')
plt.xlabel(f"Number of prior diagnoses")
plt.ylabel(f"Predicted RMST")
plt.legend()
plt.savefig("SurvivEHR.png")